## TensorFlow: Single & Multiple Hidden Layer NNs

In [1]:
import tensorflow as tf
import numpy as np

D:\Users\snandan.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Create Toy Dataset

In [2]:
np.random.seed(123)

Xtr = np.random.normal(0, 5, (2500, 3))
Xts = np.random.normal(0, 5, (2500, 3))

def f(x):
    y = (x[:, 1] + 2*x[:, 2] - x[:, 1]  # linear with no interactions
        + x[:, 0]*x[:, 1] + x[:, 0]*x[:, 1]*x[:, 2]  # linear with interactions
#          + x[:, 1]*(x[:, 2]**2)  # quadratic with interactions
        )
    return y.reshape(-1,1)

ytr = f(Xtr)
yts = f(Xts)

### Create Model

In [3]:
# hyper parameters
learning_rate = 0.01
n_input = 3
n_nodes_1 = 32
n_nodes_2 = 32
n_output = 1
batch_size = 100

# create placeholders for graph input
# the 'None' allows us to pass different batch sizes
X = tf.placeholder('float32', [None, n_input])
Y = tf.placeholder('float32', [None, n_output])


def create_weights(shape):
    # create layer weights as a variable
    # use random_normal initializer and keep the values small by setting
    # the standard devaition to 0.1 (default is 1.0)
    initializer=tf.random_normal(shape, stddev=0.1)
    W = tf.Variable(initializer)
    return W
    
def create_biases(shape):
    # create the bias parameter as varaible
    # should be rank 1 tensor with shape same as n_nodes of layer
    initializer=tf.random_normal(shape)
    b = tf.Variable(initializer)
    return b



# assign the weights we want for this model
weights = {'w1': create_weights([n_input, n_nodes_1]),  # first and only hidden layer weights
           'w2': create_weights([n_nodes_1, n_nodes_2]),
           'w_out': create_weights([n_nodes_2, n_output])}  # output layer weights

# assign biases
biases = {'b1': create_biases([n_nodes_1]),
          'b2': create_biases([n_nodes_2]),
          'b_out': create_biases([n_output])}



# Define the network forward propogation

# hidden layer
z1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])  # argument for the activation function
a1 = tf.nn.sigmoid(z1)  # the activation function

z2 = tf.add(tf.matmul(a1, weights['w2']), biases['b2'])  # argument for the activation function
a2 = tf.nn.sigmoid(z2)  # the activation function

# output layer
logits = tf.add(tf.matmul(a2, weights['w_out']), biases['b_out'])  # operates on previous layer outputs
# yhat = tf.nn.softmax(logits)  # gives class probabilities


# Back propogation
# define loss function
# RMSE
loss = tf.sqrt(tf.reduce_mean(tf.square(Y-logits)))

# define optimizer
# optimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimize = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)


# create session and initialize variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


# train over 300 epochs
n_epochs = 300
for epoch in range(1, n_epochs + 1):
    # train on one batch at a time
    for i in range(0, len(Xtr), batch_size):
        sess.run(optimize, feed_dict={X: Xtr[i: i+batch_size],
                                      Y: ytr[i: i+batch_size]})
    
    # compute training loss for printing progress
    if (epoch%10 == 0) | (epoch == 1):
        loss_tr = sess.run(loss, feed_dict={X: Xtr, Y: ytr})
        print('Epoch {}, loss: {:.3f}'.format(epoch, loss_tr, 3))
        

# compute loss for test data
loss_ts = sess.run(loss, feed_dict={X: Xts, Y: yts})
print(10*'-')
print('Test loss: {:.3f}'.format(np.round(loss_ts, 3)))

Epoch 1, loss: 115.923
Epoch 10, loss: 109.573
Epoch 20, loss: 94.345
Epoch 30, loss: 83.598
Epoch 40, loss: 74.522
Epoch 50, loss: 67.268
Epoch 60, loss: 61.372
Epoch 70, loss: 55.840
Epoch 80, loss: 50.950
Epoch 90, loss: 47.101
Epoch 100, loss: 44.124
Epoch 110, loss: 41.542
Epoch 120, loss: 40.082
Epoch 130, loss: 37.526
Epoch 140, loss: 36.018
Epoch 150, loss: 34.486
Epoch 160, loss: 32.899
Epoch 170, loss: 31.500
Epoch 180, loss: 30.220
Epoch 190, loss: 29.144
Epoch 200, loss: 28.150
Epoch 210, loss: 27.063
Epoch 220, loss: 26.215
Epoch 230, loss: 25.122
Epoch 240, loss: 25.485
Epoch 250, loss: 23.692
Epoch 260, loss: 23.205
Epoch 270, loss: 22.628
Epoch 280, loss: 22.550
Epoch 290, loss: 22.927
Epoch 300, loss: 21.990
----------
Test loss: 35.084


In [4]:
abs(yts).mean()

69.50444512983351

In [5]:
abs(ytr).mean()

62.443766639095635